## Ucitavanje i Sredjivanje Nizova Podataka

In [1]:
#ucitavanje potrebnih biblioteka
import pandas as pd
import numpy as np

In [2]:
#uvoz podataka
#sezona 23/24
df_23_4 = pd.read_csv("bundesliga-2023.csv")
#sezona 24/25
df_24_5 = pd.read_csv("bundesliga-2024.csv")
#sezona 25/6
df_25_6 = pd.read_csv("bundesliga-2025.csv")

## Ukidanje Duplih Vrijednosti za Nazive Klubova

In [3]:
#za sezonu 25 6
df_23_4["Home Team"] = df_23_4["Home Team"].replace("Sport-Club Freiburg", 'SC Freiburg')
df_23_4["Away Team"] = df_23_4["Away Team"].replace("Sport-Club Freiburg", 'SC Freiburg')
#za sezonu 24 5
df_25_6["Home Team"] = df_25_6["Home Team"].replace("Sport-Club Freiburg", 'SC Freiburg')
df_25_6["Away Team"] = df_25_6["Away Team"].replace("Sport-Club Freiburg", 'SC Freiburg')

## Određivanje Duzine Trenutnoj Sezoni i Prikaz

In [4]:
#odrediti duzinu sezone - OBAVEZNO!!!
df_25_6 = df_25_6.head(108)
df_25_6.tail()

,Match Number,Round Number,Date,Location,Home Team,Away Team,Result
103,108,12,29/11/2025 14:30,Allianz Arena,FC Bayern München,FC St. Pauli,3 - 1
104,107,12,29/11/2025 17:30,BayArena,Bayer 04 Leverkusen,Borussia Dortmund,1 - 2
105,100,12,30/11/2025 14:30,Volksparkstadion,Hamburger SV,VfB Stuttgart,2 - 1
106,106,12,30/11/2025 16:30,Deutsche Bank Park,Eintracht Frankfurt,VfL Wolfsburg,1 - 1
107,105,12,30/11/2025 18:30,Europa-Park Stadion,SC Freiburg,1. FSV Mainz 05,4 - 0


In [5]:
#sjedinjavanje ta tri df niza podataka - po osi jedan - odnosno uzduž
df_BL = pd.concat([df_23_4, df_24_5, df_25_6], axis=0)

In [6]:
#prikaz sjedinjenog niza podatak 
#tj. sve tri sezone u jednom nizu podataka u obliku df 
df_BL.tail()

,Match Number,Round Number,Date,Location,Home Team,Away Team,Result
103,108,12,29/11/2025 14:30,Allianz Arena,FC Bayern München,FC St. Pauli,3 - 1
104,107,12,29/11/2025 17:30,BayArena,Bayer 04 Leverkusen,Borussia Dortmund,1 - 2
105,100,12,30/11/2025 14:30,Volksparkstadion,Hamburger SV,VfB Stuttgart,2 - 1
106,106,12,30/11/2025 16:30,Deutsche Bank Park,Eintracht Frankfurt,VfL Wolfsburg,1 - 1
107,105,12,30/11/2025 18:30,Europa-Park Stadion,SC Freiburg,1. FSV Mainz 05,4 - 0


## Predprocesovanje Niza Podataka

In [7]:
#Izbacivanje Nerelevantnih Osobina
df_BL = df_BL.drop(["Match Number","Round Number","Date","Location"],axis=1)
df_BL.shape

(720, 3)

In [8]:
df_BL.rename(columns={"Home Team":"H","Away Team":"A"},inplace=True)

In [9]:
df_BL.to_csv("bundesliga.csv")
df_BL

,H,A,Result
0,SV Werder Bremen,FC Bayern München,0 - 4
1,VfB Stuttgart,VfL Bochum 1848,5 - 0
2,FC Augsburg,Borussia Mönchengladbach,4 - 4
3,TSG Hoffenheim,SC Freiburg,1 - 2
4,VfL Wolfsburg,1. FC Heidenheim 1846,2 - 0
...,...,...,...
103,FC Bayern München,FC St. Pauli,3 - 1
104,Bayer 04 Leverkusen,Borussia Dortmund,1 - 2
105,Hamburger SV,VfB Stuttgart,2 - 1
106,Eintracht Frankfurt,VfL Wolfsburg,1 - 1


In [10]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import pickle

#One Hot Encoder Timova - Domacin i Gost
onehot_encoder_domacin = OneHotEncoder(handle_unknown="ignore",drop="first")  # Ensure parameters are correct
kodiran_domacin = onehot_encoder_domacin.fit_transform(df_BL[["H"]]).toarray()
df_domacin = pd.DataFrame(kodiran_domacin,columns=onehot_encoder_domacin.get_feature_names_out(["H"]))

In [11]:
df_domacin.columns

Index(['H_1. FC Köln', 'H_1. FC Union Berlin', 'H_1. FSV Mainz 05',
       'H_Bayer 04 Leverkusen', 'H_Borussia Dortmund',
       'H_Borussia Mönchengladbach', 'H_Eintracht Frankfurt', 'H_FC Augsburg',
       'H_FC Bayern München', 'H_FC St. Pauli', 'H_Hamburger SV',
       'H_Holstein Kiel', 'H_RB Leipzig', 'H_SC Freiburg', 'H_SV Darmstadt 98',
       'H_SV Werder Bremen', 'H_TSG Hoffenheim', 'H_VfB Stuttgart',
       'H_VfL Bochum 1848', 'H_VfL Wolfsburg'],
      dtype='object')

In [12]:
df_domacin.shape

(720, 20)

In [13]:
#One Hot Encoder Timova - Gosti
onehot_encoder_gost = OneHotEncoder(handle_unknown="ignore",drop="first")  # Ensure parameters are correct
kodiran_gost = onehot_encoder_gost.fit_transform(df_BL[["A"]]).toarray()
df_gost = pd.DataFrame(kodiran_gost,columns=onehot_encoder_gost.get_feature_names_out(["A"]))

In [14]:
df_gost.shape

(720, 20)

In [15]:
#Sjedinjavanje tih osobina domacin i gost
df_jedan = pd.concat([df_domacin,df_gost],axis=1)
df_jedan

,H_1. FC Köln,H_1. FC Union Berlin,H_1. FSV Mainz 05,H_Bayer 04 Leverkusen,H_Borussia Dortmund,H_Borussia Mönchengladbach,H_Eintracht Frankfurt,H_FC Augsburg,H_FC Bayern München,H_FC St. Pauli,...,A_Hamburger SV,A_Holstein Kiel,A_RB Leipzig,A_SC Freiburg,A_SV Darmstadt 98,A_SV Werder Bremen,A_TSG Hoffenheim,A_VfB Stuttgart,A_VfL Bochum 1848,A_VfL Wolfsburg
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
716,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
717,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
718,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


## Transformisanje Rezultata u Varijablu - Y Varijable

#### Stvaranje Vrijednosti za Dvije Kolone iz Jedne Osobine/Kolone Rezultat

In [16]:
rez_dom = df_BL["Result"].str.slice(0,1)

In [17]:
rez_dom.head()

0    0
1    5
2    4
3    1
4    2
Name: Result, dtype: object

In [18]:
rez_gost = df_BL["Result"].str.slice(4,5)

In [19]:
rez_gost.head()

0    4
1    0
2    4
3    2
4    0
Name: Result, dtype: object

In [20]:
df_rezultatski = pd.concat([df_BL,rez_dom,rez_gost],axis=1)

In [21]:
#promjena naziva kolonama - spajanjem smo dobili kolone istih naziva
df_rezultatski.columns = ["H","A","Result","Rez1","Rez2"]
df_rezultatski

,H,A,Result,Rez1,Rez2
0,SV Werder Bremen,FC Bayern München,0 - 4,0,4
1,VfB Stuttgart,VfL Bochum 1848,5 - 0,5,0
2,FC Augsburg,Borussia Mönchengladbach,4 - 4,4,4
3,TSG Hoffenheim,SC Freiburg,1 - 2,1,2
4,VfL Wolfsburg,1. FC Heidenheim 1846,2 - 0,2,0
...,...,...,...,...,...
103,FC Bayern München,FC St. Pauli,3 - 1,3,1
104,Bayer 04 Leverkusen,Borussia Dortmund,1 - 2,1,2
105,Hamburger SV,VfB Stuttgart,2 - 1,2,1
106,Eintracht Frankfurt,VfL Wolfsburg,1 - 1,1,1


In [22]:
#spremanje df_epl varijable za obradu osobine Result u excelu
df_rezultatski.to_csv("uredjivanje_skora_vani.csv")

## Uocavanje Indeksa Sezona - za Kasnije Floatiranje

In [23]:
#df_rezultatski[303:306] #sezona 23/4 od 0:306
#df_rezultatski[306:612] #sezona 24/5 od 306:612
#df_rezultatski[612:] #sezona25/6 od 612:

## Stvaranje Kolone Skor sa Vrijednostima - Ovdje

In [24]:
# Define a function to perform the conditional logic
def calculate_value(row):
    if row['Rez1'] > row['Rez2']:
        return 1
    elif row['Rez1'] < row['Rez2']:
        return -1
    else:  # row['Column1'] == row['Column2']
        return 0

In [25]:
# Apply the function to each row in the DataFrame and assign the result to a new column
df_rezultatski['Skor'] = df_rezultatski.apply(calculate_value, axis=1)
skor = df_rezultatski.drop(['H', 'A', 'Result', 'Rez1',"Rez2"],axis=1)

### Floatiranje Vrijednosti Kolone Skor - po Sezonama

In [26]:
#sezona 23/4
skor23_4 = skor[0:306]
skor24_5 = skor[306:612]
skor25_6 = skor[612:]

In [27]:
#skor.head(3)
skor23_4["Skor"] = skor23_4["Skor"] * 0.71

C:\Users\amirb\AppData\Local\Temp\ipykernel_7716\3544292283.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  skor23_4["Skor"] = skor23_4["Skor"] * 0.71


In [28]:
skor24_5["Skor"] = skor24_5["Skor"] * 0.85

C:\Users\amirb\AppData\Local\Temp\ipykernel_7716\3136881136.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  skor24_5["Skor"] = skor24_5["Skor"] * 0.85


## Konačna kolona Skor - Sjedinjavanje Vrijednosti Sezona

In [29]:
skor = pd.concat([skor23_4, skor24_5, skor25_6], axis=0, ignore_index=True)

In [30]:
df_rezultatski = df_rezultatski.reset_index(drop=True)
skor = skor.reset_index(drop=True)

In [31]:
#provjera rezultata
finalna_provjera = pd.concat([df_rezultatski,skor],axis=1,ignore_index=True)
finalna_provjera = finalna_provjera.drop([3,4,5],axis=1)
finalna_provjera.columns = ["H","A","Rezultat","Skor"]

In [32]:
finalna_provjera.tail()

,H,A,Rezultat,Skor
715,FC Bayern München,FC St. Pauli,3 - 1,1.0
716,Bayer 04 Leverkusen,Borussia Dortmund,1 - 2,-1.0
717,Hamburger SV,VfB Stuttgart,2 - 1,1.0
718,Eintracht Frankfurt,VfL Wolfsburg,1 - 1,0.0
719,SC Freiburg,1. FSV Mainz 05,4 - 0,1.0


# Konačni df - Podaci su Sređeni

In [33]:
skor = skor.reset_index(drop=True)
df_jedan = df_jedan.reset_index(drop=True)

In [34]:
df_BL = pd.concat([df_jedan,skor],axis=1)

In [35]:
df_BL.head(5)

,H_1. FC Köln,H_1. FC Union Berlin,H_1. FSV Mainz 05,H_Bayer 04 Leverkusen,H_Borussia Dortmund,H_Borussia Mönchengladbach,H_Eintracht Frankfurt,H_FC Augsburg,H_FC Bayern München,H_FC St. Pauli,...,A_Holstein Kiel,A_RB Leipzig,A_SC Freiburg,A_SV Darmstadt 98,A_SV Werder Bremen,A_TSG Hoffenheim,A_VfB Stuttgart,A_VfL Bochum 1848,A_VfL Wolfsburg,Skor
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.71
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.71
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.71
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.71


In [36]:
#spremanje kodiranog niza podataka
df_BL.to_csv("kodirani_bl.csv")

## Razdvajanje Niza Podataka na Osobine za Predvidjanje i Ciljanu Osobinu

In [37]:
# X predvidjacke, y varijabla jeste Skor Osobina
X = df_BL.drop(["Skor"],axis=1)
y = df_BL["Skor"]

In [38]:
#razdvajanje podataka u treniranju i testirajuce nizove
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.1,random_state=42)
X_train.head()

,H_1. FC Köln,H_1. FC Union Berlin,H_1. FSV Mainz 05,H_Bayer 04 Leverkusen,H_Borussia Dortmund,H_Borussia Mönchengladbach,H_Eintracht Frankfurt,H_FC Augsburg,H_FC Bayern München,H_FC St. Pauli,...,A_Hamburger SV,A_Holstein Kiel,A_RB Leipzig,A_SC Freiburg,A_SV Darmstadt 98,A_SV Werder Bremen,A_TSG Hoffenheim,A_VfB Stuttgart,A_VfL Bochum 1848,A_VfL Wolfsburg
81,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
382,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
148,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
581,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [39]:
#skaliranje ovih osobina
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_train

array([[-0.18746672, -0.24608223, -0.24253563, ..., -0.24958983,
         4.46130315, -0.24958983],
       [-0.18746672, -0.24608223, -0.24253563, ..., -0.24958983,
         4.46130315, -0.24958983],
       [-0.18746672, -0.24608223, -0.24253563, ..., -0.24958983,
        -0.22414975, -0.24958983],
       ...,
       [-0.18746672, -0.24608223, -0.24253563, ..., -0.24958983,
        -0.22414975, -0.24958983],
       [-0.18746672, -0.24608223, -0.24253563, ..., -0.24958983,
        -0.22414975, -0.24958983],
       [-0.18746672, -0.24608223, -0.24253563, ..., -0.24958983,
        -0.22414975, -0.24958983]])

In [40]:
#Spremanje tog Kodera - za Kasniju Upotrebu

with open("onehot_encoder_domacin.pkl","wb") as file:
    pickle.dump(onehot_encoder_domacin,file)
    
with open("onehot_encoder_gost.pkl","wb") as file:
    pickle.dump(onehot_encoder_gost,file)
    
with open("scaler.pkl","wb") as file:
    pickle.dump(scaler,file)
    

## Stvaranje Modela i Dobijanje Predikcije

In [41]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
#sledeca biblioteka se odnosi na stvaranje neurona iz kutije layers i alatom dense
from tensorflow.keras.layers import Dense

In [42]:
#konstrukcija modela
model=Sequential([
    Dense(16,activation="relu",input_shape=(X_train.shape[1],)), #HL1 connected to I/p - Skriveni Sloj 1 povezan sa I/p
    Dense(1,activation="tanh")
])

In [43]:
#Oblikovanje Modela 
model.compile(optimizer="adam",loss="mean_absolute_error",metrics=["mae"])
model.summary()

# Keras dokumentacija pojasnjava sta se dogodi sa razlicitim odabirom "loss" i drugih parametara

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 16)                656       
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 673
Trainable params: 673
Non-trainable params: 0
_________________________________________________________________


In [44]:
import datetime
#uspostavljanje Tensorboard-a, zapocinjanje treniranja
#TensorBoard sluzi samo za vizualizaciju svih logovanja koje izvrsim kod treniranja nekog modela
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard

#log_dir varijabla koja osigurava da ostane zabinjezen trag svakog treniranja modela
log_dir = "regressionlogs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir=log_dir,histogram_freq=1)

#Uspostavljanje Early Stopping - sluzi za zaustavljanje treniranja ako se vrijednost gubitaka ne smanjuje
#bitni parametri su uneseni koji sluze poboljsanju funkcije ranog zaustavljanja treniranja modela...
early_stopping_callback = EarlyStopping(monitor="val_loss",patience=10,restore_best_weights=True)


In [45]:
#TRENIRANJE MODELA / Tovarenje Modela
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard

history = model.fit(
    X_train,y_train,validation_data=(X_test,y_test),epochs=50,
    callbacks= [tensorflow_callback,early_stopping_callback]
)

Epoch 1/50
21/21 [==============================] - 7s 99ms/step - loss: 0.7332 - mae: 0.7332 - val_loss: 0.7147 - val_mae: 0.7147
Epoch 2/50
21/21 [==============================] - 1s 26ms/step - loss: 0.6916 - mae: 0.6916 - val_loss: 0.6932 - val_mae: 0.6932
Epoch 3/50
21/21 [==============================] - 0s 22ms/step - loss: 0.6563 - mae: 0.6563 - val_loss: 0.6768 - val_mae: 0.6768
Epoch 4/50
21/21 [==============================] - 0s 22ms/step - loss: 0.6222 - mae: 0.6222 - val_loss: 0.6628 - val_mae: 0.6628
Epoch 5/50
21/21 [==============================] - 0s 22ms/step - loss: 0.5945 - mae: 0.5945 - val_loss: 0.6491 - val_mae: 0.6491
Epoch 6/50
21/21 [==============================] - 1s 28ms/step - loss: 0.5687 - mae: 0.5687 - val_loss: 0.6333 - val_mae: 0.6333
Epoch 7/50
21/21 [==============================] - 0s 24ms/step - loss: 0.5472 - mae: 0.5472 - val_loss: 0.6183 - val_mae: 0.6183
Epoch 8/50
21/21 [==============================] - 0s 22ms/step - loss: 0.5289 - m

In [46]:
#Procjena Rada Modela preko test podataka
test_loss, test_mae = model.evaluate(X_test,y_test)

3/3 [==============================] - 0s 5ms/step - loss: 0.5755 - mae: 0.5755


In [47]:
#spremanje modela
model.save("BL_float_model.h5")

In [48]:
#moze biti da skaliranje je neophodno, onda je potrebno i obnoviti test parametara modela, te iznova
#-trenirati model i na kraju ga ponovo spremiti.